In [111]:
import array
import os
#import commands
import time
import math
import sys
import string
import fileinput
import re
import subprocess
import traceback
from typing import List, Tuple, Union

In [112]:
def find_float(arr0, st0, fl0):
    for line in arr0:
        find = re.search(st0, line)
        if (find):
            t = re.search('-*[0-9]*\.*[0-9]+', line)
            if t is not None:
                fl0 = float(t.group())
                return 1, fl0
    return -1, fl0

In [113]:
def search_best(arr0, st0, st1, ab, minmax, fl0):
    if minmax == "min":
        fl0 = 1e+20
    else:
        fl0 = -1e+20
    for line in arr0:
        find = re.search(st0, line)
        if (find):
            find = re.search(st1, line)
            if (find):
                split_line = line.split(' ')
                st1_split = st1.split(' ')
                words = len(st1_split)
                i = 0
                for word in split_line:
                    if word == st1_split[i]:
                        i += 1
                    if i == words:
                        if ab == "a":
                            ind = split_line.index(word) + 2
                            fl1 = split_line[ind]
                            if "inf" in fl1:
                                fl1 = 1e+20 if minmax == "min" else -1e+20
                            else:
                                fl1 = float(fl1)
                            if minmax == "min" and fl1 < fl0:
                                fl0 = fl1
                            elif minmax == "max" and fl1 > fl0:
                                fl0 = fl1
                        else:
                            ind = split_line.index(word) - words
                            fl1 = float(split_line[ind])
                            if minmax == "min" and fl1 < fl0:
                                fl0 = fl1
                            elif minmax == "max"  and fl1 > fl0:
                                fl0 = fl1
                        break
    return 1, fl0

In [114]:
def InstanceOutput(instance,output):
    file_data = output.read().split('\n')
    time = 0
    find, time = find_float(file_data, "MultiStart: wall clock time used", time)
    if find < 0:
        time = 3600
        
    status = ""
    for line in file_data:
        find = re.search("MultiStart: status of branch-and-bound:", line)
        if (find):
                status = line.partition("MultiStart: status of branch-and-bound: ")[2]
                if status == "Optimal solution found":
                    status = "OPT"
                elif status == "Reached time limit":
                    status = "TL"
                elif status == "Detected infeasibility":
                    status = "INFEAS"
                else:
                    print(instance, "THIS SHOULD NOT HAPPEN")
                    sys.exit(0)
    ##UB
    ub = 0
    if status == "":
        find, ub = search_best(file_data, "BranchAndBound:", "ub", "a", "min", ub)
        if ub == 1e+20:
            ub = "INFTY"
    elif status == "INFEAS":
            ub = "INFEAS"
    else:
        find, ub = find_float(file_data, "MultiStart: best solution value", ub)

    ##LB
    lb = 0
    if status == "":
        find, lb = search_best(file_data, "BranchAndBound:", "lb", "a", "max", lb)
        if lb == -1e+20:
            lb = "-INFTY"
    elif status == "INFEAS":
        lb = "INFEAS"
    elif status == "OPT":
        lb = ub
    else:
        find, lb = find_float(file_data, "MultiStart: best bound estimate from remaining nodes", lb)

    ##Gap
    gap = 0
    if status == "":
        find, gap = search_best(file_data, "BranchAndBound:", "gap%", "a", "min", gap)
        if gap == 1e+20:
            gap = "INFTY"
    elif status == "INFEAS":
            gap = "INFEAS"
    elif status == "OPT":
            gap = 0.00
    else:
        find, gap = find_float(file_data, "MultiStart: gap percentage =", gap)
    
    if status == "":
        status = "TL"
    
    # print("model instance:",instance)
    # print("Objective:",ub)
    # print("Time:",time) 
    return ub, time,ub

In [115]:
def BaronInstanceOutput(instance, output):
    file_data = output.read().split('\n')
    time = 0
    find, time = find_float(file_data, "Total CPU time used: ", time)
    Objective = 0
    find, Objective = find_float(file_data, "Objective ", Objective)
    # Objective = 0
    
    for line in file_data:
        find1 = re.search("Problem is infeasible", line)
        if (find1):
            print("Problem is infeasible")
            Objective = "Infeasible "
        find2 = re.search("No feasible solution was found", line)
        if (find2):
            print("No feasible solution was found")
            Objective = 1e+51

    # print("model instance:",instance)
    # print("Objective:",Objective)
    # print("Lower bound:",lb1)
    # print("Uper bound:",ub1)
    # print("Time:",time) 
    # print(" ")
    return Objective, time

In [116]:
def KnitroInstanceOutput(instance, output):
    file_data = output.read().split('\n')
    time = 0
    find, time = find_float(file_data, "Total program time", time)
    Objective = 0
    find, Objective = find_float(file_data, "objective  ", Objective)
    # Objective = 0
    FEA_ERR =0 
    for line in file_data:
        find1 = re.search("objective", line)
        if (find1):
            Objective = line.partition("objective")[2]
            # print(Objective)
            Objective = Objective.partition(";")[0]
            # FEA_ERR= line.partition("feasibility error")[2]
            # print(FEA_ERR)
            # a,b = FEA_ERR.split("e")
            # FEA_ERR = float(a)*(10**float(b))
            # print(FEA_ERR)
            # FEA_ERR = float(FEA_ERR)
            # if FEA_ERR >0:
            #     Objective = "infeasible"

    # print("model instance:",instance)
    # print("Objective:",Objective)
    # print("Lower bound:",lb1)
    # print("Uper bound:",ub1)
    # print("Time:",time) 
    # print(" ")
    return Objective, time

In [117]:
data_list=["d1_Sample_input_cycle_twoloop",
           "d2_Sample_input_cycle_hanoi",
           "d3_Sample_input_double_hanoi",
           "d4_Sample_input_triple_hanoi",
           "d5_Taichung_input",
           "d6_HG_SP_1_4",
           "d7_HG_SP_2_3",
           "d8_HG_SP_3_4",
           "d9_HG_SP_4_2",
           "d10_HG_SP_5_5",
           "d11_HG_SP_6_3",
           "d12",
           "d13",
           "d14_NewYork",
           "d15_foss_poly_0",
           "d16_foss_iron",
           "d17_foss_poly_1",
           "d18_pescara",
           "d19_modena"]

for ins in data_list:
    with open(f"../output/knitro_out/1h/{ins}.knitro_out") as output:
        print("Model Name:",ins)
        obj, time = KnitroInstanceOutput(ins,output)
        print("Objective :",obj)
        print("Time :",time)
        print(" ")
        # Knitro_Objective.append(obj)
        # Knitro_Time_taken.append(time)

Model Name: d1_Sample_input_cycle_twoloop
Objective :  403606.7874
Time : 26.98417
 
Model Name: d2_Sample_input_cycle_hanoi
Objective :  6062283.475
Time : 24.75909
 
Model Name: d3_Sample_input_double_hanoi
Objective :  12080703.89
Time : 65.91277
 
Model Name: d4_Sample_input_triple_hanoi
Objective :  18646209.08
Time : 153.19084
 
Model Name: d5_Taichung_input
Objective :  8846230.934
Time : 25.1545
 
Model Name: d6_HG_SP_1_4
Objective :  3567636.841
Time : 122.45294
 
Model Name: d7_HG_SP_2_3
Objective :  3402071.259
Time : 113.56051
 
Model Name: d8_HG_SP_3_4
Objective :  6920428.474
Time : 1148.4668
 
Model Name: d9_HG_SP_4_2
Objective :  8910504.673
Time : 699.19836
 
Model Name: d10_HG_SP_5_5
Objective :  9377732.869
Time : 348.97073
 
Model Name: d11_HG_SP_6_3
Objective :  10707680.41
Time : 336.25464
 
Model Name: d12
Objective :  3413417.149
Time : 100.25037
 
Model Name: d13
Objective :  14902767.47
Time : 6.79915
 
Model Name: d14_NewYork
Objective :  232835065.9
Time : 2

In [122]:
def IpoptInstanceOutput(instance, output):
    file_data = output.read().split('\n')
    time = 0
    find, time = find_float(file_data, "Total seconds in IPOPT", time)
    Objective = 0
    find, Objective = find_float(file_data, "total_cost", Objective)
    return Objective, time

In [119]:
import numpy as np 

In [120]:
def BonminInstanceOutput(instance, output):
    file_data = output.read().split('\n')
    time = 0
    # find, time = find_float(file_data, "Total CPU secs in IPOPT", time)
    Objective = 0
    find, Objective = find_float(file_data, "total_cost", Objective)
    T = []
    for line in file_data:
        find1 = re.search("NLP0014I", line)
        if (find1):
            find2 = re.search("OPT",line)
            if find2:
                time = line.partition("OPT")[2]
                time = str(time).split()[2]
                time = float(time)
                # print(time)
                T.append(time)

            find2 = re.search("INFEAS",line)
            if find2:
                time = line.partition("INFEAS")[2]
                time = str(time).split()[2]
                time = float(time)
                T.append(time)
                # print(time)

                Objective = "INFEAS"
    time = sum(T)
    return Objective, time

In [123]:
data_list=["d1_Sample_input_cycle_twoloop",
           "d2_Sample_input_cycle_hanoi",
           "d3_Sample_input_double_hanoi",
           "d4_Sample_input_triple_hanoi",
           "d5_Taichung_input",
           "d6_HG_SP_1_4",
           "d7_HG_SP_2_3",
           "d8_HG_SP_3_4",
           "d9_HG_SP_4_2",
           "d10_HG_SP_5_5",
           "d11_HG_SP_6_3",
           "d12",
           "d13",
           "d14_NewYork",
           "d15_foss_poly_0",
           "d16_foss_iron",
           "d17_foss_poly_1",
           "d18_pescara",
           "d19_modena"]

# !python3 ampl_run.py m1_basic.mod d1_Sample_input_cycle_twoloop.dat > output/d1_Sample_input_cycle_twoloop.baro_out

import pandas as pd
import csv

Ins = []
Mmultistart_Objective = []
Mmultistart_Time_taken = []
Baron_Objective = []
Baron_Time_taken = []
Knitro_Objective = []
Knitro_Time_taken = []
Ipopt_Objective = []
Ipopt_Time_taken = []
Bonmin_Objective = []
Bonmin_Time_taken = []

print("****************************Results of Mmultistart Solver *********************************")

for ins in data_list:
    with open(f"../output/m_out/1h/{ins}.m_out") as output:
        print("Model Name:",ins)
        obj, time,ub = InstanceOutput(ins,output)
        print("Objective :",obj)
        print("Time :",time)
        print(" ")
        Ins.append(ins)
        Mmultistart_Objective.append(obj)
        Mmultistart_Time_taken.append(time)

print("******************************Results of Baron Solver ************************************")

for ins in data_list:
    with open(f"../output/baron_out/1h/{ins}.baron_out") as output:
        print("Model Name:",ins)
        obj, time= BaronInstanceOutput(ins,output)
        print("Objective :",obj)
        print("Time :",time)
        print(" ")
        Baron_Objective.append(obj)
        Baron_Time_taken.append(time)

print("**********************Results of Knitro Solver *********************************")
 
for ins in data_list:
    with open(f"../output/knitro_out/1h/{ins}.knitro_out") as output:
        print("Model Name:",ins)
        obj, time = KnitroInstanceOutput(ins,output)
        print("Objective :",obj)
        print("Time :",time)
        print(" ")
        Knitro_Objective.append(obj)
        Knitro_Time_taken.append(time)

print("**********************Results of IPOPT Solver *********************************")

for ins in data_list:
    with open(f"../output/ipopt_out/1h/{ins}.ipopt_out") as output:
        print("Model Name:",ins)
        obj, time = IpoptInstanceOutput(ins,output)
        print("Objective :",obj)
        print("Time :",time)
        print(" ")
        Ipopt_Objective.append(obj)
        Ipopt_Time_taken.append(time)
print("**********************Results of BONMIN Solver *********************************")
        
for ins in data_list:
    with open(f"../output/bonmin_out/1h/{ins}.bonmin_out") as output:
        print("Model Name:",ins)
        obj, time = BonminInstanceOutput(ins,output)
        print("Objective :",obj)
        print("Time :",time)
        print(" ")
        Bonmin_Objective.append(obj)
        Bonmin_Time_taken.append(time)
# Solver_name = [" ","mmultistart", " ", "Baron", " ", "Knitro "," "]

fields = ["Instances","Mmultistart Objective","Mmultistart time taken","Baron Objective","Baron time taken", "Knitro Objective","Knitro time taken","Ipopt Objective","Ipopt time taken", "Bonmin Objective","Bonmin time taken" ]

filename = "mmultistart_results.csv"

with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    # csvwriter.writerow(Solver_name)
    csvwriter.writerow(fields)

import pandas as pd
csv_input = pd.read_csv(filename)
csv_input['Instances'] = Ins
csv_input['Mmultistart Objective'] = Mmultistart_Objective
csv_input['Mmultistart time taken'] = Mmultistart_Time_taken
csv_input['Baron Objective'] = Baron_Objective
csv_input['Baron time taken'] = Baron_Time_taken
csv_input['Knitro Objective'] = Knitro_Objective
csv_input['Knitro time taken'] = Knitro_Time_taken
csv_input['Ipopt Objective'] = Ipopt_Objective
csv_input['Ipopt time taken'] = Ipopt_Time_taken
csv_input['Bonmin Objective'] = Bonmin_Objective
csv_input['Bonmin time taken'] = Bonmin_Time_taken

csv_input.to_csv('output.csv', index=False)

df = pd.read_csv("output.csv")

excel_file = pd.ExcelWriter('output.xlsx')
df.to_excel(excel_file, index = False)

excel_file.save()
excel_file = pd.ExcelWriter('output.xlsx')

****************************Results of Mmultistart Solver *********************************
Model Name: d1_Sample_input_cycle_twoloop
Objective : 403606.7874
Time : 19.34
 
Model Name: d2_Sample_input_cycle_hanoi
Objective : 6064901.0669
Time : 18.09
 
Model Name: d3_Sample_input_double_hanoi
Objective : 12071816.8568
Time : 42.94
 
Model Name: d4_Sample_input_triple_hanoi
Objective : 18388458.7195
Time : 97.9
 
Model Name: d5_Taichung_input
Objective : 9213593.9371
Time : 33.39
 
Model Name: d6_HG_SP_1_4
Objective : 3655248.6372
Time : 638.87
 
Model Name: d7_HG_SP_2_3
Objective : INFTY
Time : 3600
 
Model Name: d8_HG_SP_3_4
Objective : 6920355.4354
Time : 550.27
 
Model Name: d9_HG_SP_4_2
Objective : INFTY
Time : 3600
 
Model Name: d10_HG_SP_5_5
Objective : INFTY
Time : 3600
 
Model Name: d11_HG_SP_6_3
Objective : INFTY
Time : 3600
 
Model Name: d12
Objective : INFTY
Time : 3600
 
Model Name: d13
Objective : 14902767.4736
Time : 4.4
 
Model Name: d14_NewYork
Objective : INFEAS
Time :

In [6]:
data_list=["d1_Sample_input_cycle_twoloop.m_out",
           "d2_Sample_input_cycle_hanoi.m_out",
           "d3_Sample_input_double_hanoi.m_out",
           "d4_Sample_input_triple_hanoi.m_out",
           "d5_Taichung_input.m_out",
           "d6_HG_SP_1_4.m_out",
           "d7_HG_SP_2_3.m_out",
           "d8_HG_SP_3_4.m_out",
           "d9_HG_SP_4_2.m_out",
           "d10_HG_SP_5_5.m_out",
           "d11_HG_SP_6_3.m_out",
           "d12.m_out",
           "d13.m_out",
           "d14_NewYork.m_out",
           "d15_foss_poly_0.m_out",
           "d16_foss_iron.m_out",
           "d17_foss_poly_1.m_out",
           "d18_pescara.m_out",
           "d19_modena.m_out"]

# !python3 ampl_run.py m1_basic.mod d1_Sample_input_cycle_twoloop.dat > output/d1_Sample_input_cycle_twoloop.baro_out

import pandas as pd
import csv

csv_data = []

for ins in data_list:
    with open(f"../output/m_out/{ins}") as output: 
        print("Model Name:",ins)
        Objective, time,ub,lb, status, gap = InstanceOutput(ins,output)
        csv_data.append([ins,Objective, time, ub, lb, gap, status])
    # print("################################## Mmultistart Solver Result #############################")        
    # print("Model Name:",ins)
    # print("Objective:",Objective)
    # print("Time:",time)

fields = ["Instances","Objective","time taken", "upper bound", "lower bound", "gap", "status"]

rows = csv_data

filename = "mmultistart_results.csv"

with open(filename, 'w') as csvfile:

    csvwriter = csv.writer(csvfile)

    csvwriter.writerow(fields)

    csvwriter.writerows(rows)

df = pd.read_csv(filename)

excel_file = pd.ExcelWriter('mmultistart_results.xlsx')
df.to_excel(excel_file, index = False)

excel_file.save()
excel_file = pd.ExcelWriter('mmultistart_results.xlsx')

Model Name: d1_Sample_input_cycle_twoloop.m_out
Objective: 471747.3702
Time: 1.83
Model Name: d2_Sample_input_cycle_hanoi.m_out
Objective: 6059622.0198
Time: 3.48
Model Name: d3_Sample_input_double_hanoi.m_out
Objective: 12071816.8568
Time: 9.91
Model Name: d4_Sample_input_triple_hanoi.m_out
Objective: 18386806.0098
Time: 19.77
Model Name: d5_Taichung_input.m_out
Objective: 9213181.4041
Time: 16.23
Model Name: d6_HG_SP_1_4.m_out
Objective: 3556551.1218
Time: 117.48
Model Name: d7_HG_SP_2_3.m_out
Objective: INFTY
Time: 3600
Model Name: d8_HG_SP_3_4.m_out
Objective: 6928757.5771
Time: 86.12
Model Name: d9_HG_SP_4_2.m_out
Objective: INFTY
Time: 3600
Model Name: d10_HG_SP_5_5.m_out
Objective: INFTY
Time: 3600
Model Name: d11_HG_SP_6_3.m_out
Objective: INFTY
Time: 3600
Model Name: d12.m_out
Objective: INFTY
Time: 3600
Model Name: d13.m_out
Objective: 14902767.4736
Time: 0.72
Model Name: d14_NewYork.m_out
Objective: INFEAS
Time: 6.84
Model Name: d15_foss_poly_0.m_out
Objective: 66455254.4783

In [1]:
import re

In [2]:
# opening the file in read mode 
my_file = open("./minlplib/new_inst_list.txt", "r") 
  
# reading the file 
data = my_file.read() 
  
# replacing end of line('/n') with ' ' and 
# splitting the text it further when '.' is seen. 
data_into_list = data.split("\n") 
  
# printing the data 
# print(data_into_list) 
my_file.close() 


In [10]:
def ErrorFind(output):
    file_data = output.read().split('\n')

    for line in file_data:
        # print(line)
        # error = re.search("IpoptEngine: error reported.", line)
        # error = re.search("IpoptEngine: error in evaluating jacobian", line)
        # error = re.search("MsProcessor: engine hit iteration limit,", line)
        # error = re.search("IpoptFunInterface: error in evaluating constraint", line)
        error = re.search("MultiStart: status of branch-and-bound: Detected infeasibility", line)
        # error = str("Assertion `!"Ipopt: search direction becomes too small."' failed.")
        # error = re.search(error, line)
        if error:
            return True
        else:
            continue
        

Ipopt_Error_list = []
for ins in data_into_list:
    with open(f"./minlplib/10_min_output/{ins}.out") as output: 
        # print("Instance Name:",ins)
        error = ErrorFind(output)
        if error == True:
            # print(ins)
            Ipopt_Error_list.append(ins)
            
len(Ipopt_Error_list)

144

In [1]:
from scipy.optimize import fsolve
import numpy as np

# Define the equation
def equation(x, b, c):
    return x**1.852 + b*x**0.852 - c

# Given values for b and c
b = 0.6762
c = 0.1350

# Initial guess for the root
initial_guess = 1.0

# Use fsolve to find the root
root = fsolve(equation, initial_guess, args=(b, c))

print("Root:", root)

Root: [0.12386961]
